In [1]:
!nvidia-smi

Thu Aug  4 06:49:20 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    42W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!unzip New_SNE_Code.zip

unzip:  cannot find or open New_SNE_Code.zip, New_SNE_Code.zip.zip or New_SNE_Code.zip.ZIP.


In [ ]:
!pip install umap
!pip install trimap
!pip3 install tsnecuda==3.0.1+cu111 -f https://tsnecuda.isx.ai/tsnecuda_stable.html


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://tsnecuda.isx.ai/tsnecuda_stable.html


In [ ]:

%load_ext autoreload
%autoreload 2
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import sys
sys.path.insert(1, '/content/drive/MyDrive/STU_Rafae_Baez_Ramirez/New_SNE_Code')
sys.path.insert(1, '/content/drive/MyDrive/STU_Rafae_Baez_Ramirez/New_SNE_Code/LAPS_and_GAPS-master')
#sys.path.insert(1, r'C:/Users/baezr/PythonCode/New_SNE_Code/')
#sys.path.insert(1, r'C:/Users/baezr/PythonCode/New_SNE_Code/LAPS_and_GAPS-master/')

## Importing libraries

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

import os
import sys
import math
import time
import numpy as np
import pandas as pd
from bokeh.io import show, output_notebook, curdoc
from bokeh.plotting import figure
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.models import HoverTool
from bokeh.io import output_notebook
from bokeh.models.glyphs import Text
from bokeh.layouts import row
from bokeh.io import export_png
output_notebook()

## Importing local Python files

# sys.path.append('../') 


Mounted at /content/drive


In [ ]:
#%cd C:\Users\baezr\PythonCode\New_SNE_Code\LAPS_and_GAPS-master\
%cd /content/drive/MyDrive/STU_Rafae_Baez_Ramirez/New_SNE_Code/LAPS_and_GAPS-master
from src import Preprocessing, Distances, Explanation, DR_algorithms, genericMethods
from src.LAPS_tabular import LapsExplainer
from src.GAPS_tabular import GapsExplainer
from src.GAPS_Explanation import get_local_explanations_for_GAPS, compute_local_divergences


/content/drive/.shortcut-targets-by-id/19cmY-8oRuw_0ZRIxg6V-cBwh6M492s7s/STU_Rafae_Baez_Ramirez/New_SNE_Code/LAPS_and_GAPS-master


In [ ]:
Data_path = '/content/drive/MyDrive/STU_Rafae_Baez_Ramirez/New_SNE_Code/LAPS_and_GAPS-master/data/'

In [ ]:
def LAPS_and_plot(embedding, data_transformed, data_df, y_res, model_feats, targ, cat_feats, cat_names, num_feats, point, show_fig = False, verbose = False):
  exp = LapsExplainer(data_df.values, feature_names=model_feats, class_names=targ, categorical_features=cat_feats, categorical_names=cat_names, discretize_continuous=False, discretizer='quartile', random_state=42)
  neigh, neigh_emb, over_data, over_data_emb = exp.generate_perturbed_neighborhood(data_df.values, data_transformed, embedding, point, data_df.values[point], nbrs=20, num_features=5, verbose = verbose)
  corr_feat_dist, _, _, _, _ = Explanation.explain_point_local(data_df.values[point], neigh, over_data, model_feats, cat_feats, num_feats)
  corr_feat_dist_embd, _, _, _, _ = Explanation.explain_point_local(data_df.values[point], neigh_emb, over_data_emb, model_feats, cat_feats, num_feats)
  components, divergence = Explanation.compute_local_divergence(corr_feat_dist, corr_feat_dist_embd, neigh, neigh_emb)
  components = components.split(",")
  if show_fig:
    idx = [i for i in range(len(y_res))]
    colormap = {-1: 'Red', 
                0: 'gold', 
                1: 'peachpuff', 
                2: 'limegreen',
                3: 'green',
                4: 'darkolivegreen',
                5: 'cyan',
                6: 'blue',
                7: 'lightsteelblue',
                8: 'magenta',
                9: 'indigo'}
    colors = [colormap[z] for z in y_res.ravel()]
    labels = {-1: 'POI', 
                0: 'Class 0', 
                1: 'Class 1', 
                2: 'Class 2',
                3: 'Class 3',
                4: 'Class 4',
                5: 'Class 5',
                6: 'Class 6',
                7: 'Class 7',
                8: 'Class 8',
                9: 'Class 9'} 
    annotations = [labels[z] for z in y_res.ravel()]
    source = ColumnDataSource(
          data=dict(
              x=embedding[:,0],
              y=embedding[:,1],
              all_colors = colors,
              label = annotations,
              indexes = idx
          )
      )
    hover = HoverTool(
          tooltips=[
              ("index", "$index"),
              ("(x,y)", "($x, $y)"),
          ]
      )
    curdoc().theme = 'dark_minimal'
    p = figure(plot_width=700, plot_height=450, tools=[hover],
            title="Embedding")
    glyph = Text(x="x", y="y", text='indexes', x_offset=7, y_offset=7, text_font_size="9pt", text_color="grey")
    p.circle('x', 'y', fill_color='all_colors', line_color='white', legend='label', size=10, source=source)
    p.add_glyph(source, glyph)
    show(p)
  return components, divergence

In [ ]:
def GAPS_score(embedding, data_df, data_transformed, model_feats, target, cat_feats, cat_names, num_feats, POI = None, show_fig = False, verbose = False):
  if POI is None:
    rep_subset = genericMethods.generate_representative_subset(18, data_df)
    data_instance_numbers = rep_subset[0]
  else:
    data_instance_numbers = POI
  #print(data_instance_numbers)
  explainer = GapsExplainer(data_df.values, feature_names=model_feats,
                            class_names=target, categorical_features=cat_feats,
                            discretize_continuous=False, discretizer='quartile',
                            random_state=42)
  neigh, neigh_emb, over_data, over_data_emb, local_feat_cont, local_feat_cont_emb, neigh_local, neigh_emb_local = explainer.generate_perturbed_neighborhood_global(
      data_df.values, data_transformed, embedding, data_instance_numbers, model_feats, cat_feats, num_feats, nbrs = 5, num_features = 5, verbose = False)
  local_divergences = compute_local_divergences(neigh_local, neigh_emb_local, local_feat_cont, local_feat_cont_emb)
  corr_feat_dist, _, _ = Explanation.explain_point_global(over_data, model_feats, cat_feats, num_feats)
  corr_feat_dist_embd, _, _ = Explanation.explain_point_global(over_data_emb, model_feats, cat_feats, num_feats)
  components, overall_divergence = Explanation.compute_global_divergence(corr_feat_dist, corr_feat_dist_embd, neigh, neigh_emb, local_divergences)
  components = components.split(',')
  return components, overall_divergence, POI

In [ ]:
percPOI = 0.10


df = pd.read_csv(Data_path+'breast_cancer.csv')
# df = pd.read_csv(Data_path+'Credit_card.csv')
print(df.shape)
df = df.fillna(0)
#df = df.sample(100)
df.reset_index(inplace= True, drop= True)
df.head()

## Identify intrinsic dimensionality using: the Maximum likelihood intrinsic dimensionality estimator

#int_dim = Preprocessing.repeated(Preprocessing.intrinsic_dim_scale_interval, df.values,mode='bootstrap', nb_iter=500, verbose=1, 
                            #  k1=10, k2=20)
#print ("Intrinsic dimensionality:", int_dim)

model_features, target = Preprocessing.set_features_and_target(df)

y = (df[target].values.reshape(-1, ))
X_df = pd.DataFrame(df, columns=model_features)
POI = np.random.randint(0, X_df.shape[0], int(X_df.shape[0]*percPOI))

# POI = [24,86]
new_y = np.copy(y)
for i in POI:
  new_y[i] = -1
print(f'POI: {POI}')
X_transformed, categorical_features, numeric_features, categorical_names = Preprocessing.identify_and_transform_features(df, model_features)
constants = [X_df, new_y, model_features, target, categorical_features, categorical_names, numeric_features, POI]

(569, 31)
Features:  ['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean', 'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean', 'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se', 'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se', 'fractal_dimension_se', 'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst', 'smoothness_worst', 'compactness_worst', 'concavity_worst', 'concave points_worst', 'symmetry_worst', 'fractal_dimension_worst']
Target:  ['diagnosis']
POI: [ 37 518 314 543 419 179 209 516 239 212 227 395 234 321 455 510 154 558
 468 529 400 405 362 188 126 107 353  12 455 194 170  47 414 173 409 532
  38 381 441  66 164 495 442 516  81 563 410 163 491 339 406  91  72 302
 428 408]


In [ ]:
all_runs = 5
iterations = 300
# %cd /content/New_SNE_Code
#%cd /content/drive/MyDrive/STU_Rafae_Baez_Ramirez/New_SNE_Code
from TorchSNE import TorchSNE
from tsnecuda import TSNE
#import fSNE_opt_mult
from fSNE_opt import fSNE
from my_fSNE import my_fSNE
from my_SNE import My_SNE

In [ ]:
!nvidia-smi

Fri Jul  1 12:57:00 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 512.95       Driver Version: 512.95       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   51C    P8     4W /  N/A |      0MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#### My_SNE

In [ ]:
SNE = My_SNE(X_transformed.T, max_iterations = iterations)
best_score_sne = 1
best_emb_sne = 0
for i in range(all_runs):
  embed_sne = SNE.fit_transform().T
  comp, div = LAPS_and_plot(embed_sne, X_transformed, 
                            constants[0], constants[1], 
                            constants[2], constants[3], 
                            constants[4], constants[5], 
                            constants[6], constants[7][0], show_fig = False, verbose = False)
  if div < best_score_sne:
    best_score_sne = div
    best_emb_sne = embed_sne
# global_comp, global_div, _ = GAPS_score(embed_sne, constants[0], X_transformed, 
#                                         constants[2], constants[3], 
#                                         constants[4], constants[5], 
#                                         constants[6], constants[7], show_fig = False)

_, div = LAPS_and_plot(best_emb_sne, X_transformed,
                constants[0], constants[1], 
                constants[2], constants[3], 
                constants[4], constants[5], 
                constants[6], constants[7][0], show_fig = True, verbose = False)
print(f'Between {all_runs} SNE\s best run got a score of {div}')

Calculating p for all pairs...
---processing for image 0
---processing for image 1
---processing for image 2
---processing for image 3
---processing for image 4
---processing for image 5
---processing for image 6
---processing for image 7
---processing for image 8
---processing for image 9
---processing for image 10
---processing for image 11
---processing for image 12
---processing for image 13
---processing for image 14
---processing for image 15
---processing for image 16
---processing for image 17
---processing for image 18
---processing for image 19
---processing for image 20
---processing for image 21
---processing for image 22
---processing for image 23
---processing for image 24
---processing for image 25
---processing for image 26
---processing for image 27
---processing for image 28
---processing for image 29
---processing for image 30
---processing for image 31
---processing for image 32
---processing for image 33
---processing for image 34
---processing for image 35
---proc

KeyboardInterrupt: ignored

#### SNE (torch)

In [ ]:
SNE = TorchSNE(verbose = True, n_iter = iterations)
#start = time.time()
best_score_sne = 1
best_emb_sne = 0
scores = []
for i in range(all_runs):
  embed_sne = SNE.fit_transform(X_transformed)
  comp, div = LAPS_and_plot(embed_sne, X_transformed, 
                            constants[0], constants[1], 
                            constants[2], constants[3], 
                            constants[4], constants[5], 
                            constants[6], constants[7][0], show_fig = False, verbose = False)
  scores.append(div)
  if div < best_score_sne:
    best_score_sne = div
    best_emb_sne = embed_sne
_, div = LAPS_and_plot(best_emb_sne, X_transformed,
                constants[0], constants[1], 
                constants[2], constants[3], 
                constants[4], constants[5], 
                constants[6], constants[7][0], show_fig = True, verbose = False)
print(f'Between {all_runs} SNE\s best run got a score of {div}')
print(f'All the scores were {scores}')

using cuda
initializing...
computing p-values...
fiting...
error: 50.703: 100%|██████████| 300/300 [00:01<00:00, 249.82it/s]
using cuda
initializing...
computing p-values...
fiting...
error: 50.703: 100%|██████████| 300/300 [00:01<00:00, 246.71it/s]
using cuda
initializing...
computing p-values...
fiting...
error: 50.703: 100%|██████████| 300/300 [00:00<00:00, 342.26it/s]
using cuda
initializing...
computing p-values...
fiting...
error: 50.703: 100%|██████████| 300/300 [00:01<00:00, 239.56it/s]
using cuda
initializing...
computing p-values...
fiting...
error: 50.703: 100%|██████████| 300/300 [00:01<00:00, 246.71it/s]


Between 5 SNE\s best run got a score of 0.8301364500372057
All the scores were [0.8301364500372057, 0.8301364500372057, 0.8301364500372057, 0.8301364500372057, 0.8301364500372057]


#### TSNE (cuda)

In [ ]:
#start = time.time()
#embed_tsne = TSNE(n_components=2, learning_rate=0.1, n_iter= iterations,verbose=True).fit_transform(X_transformed)
#print(f'That took {(time.time()-start):.4f} seconds')
best_score_tsne = 1
best_emb_tsne = 0
scores = []
for i in range(all_runs):
  embed_tsne = TSNE(n_components=2, learning_rate=0.1, n_iter= iterations,verbose=True).fit_transform(X_transformed)
  _, div = LAPS_and_plot(embed_tsne, X_transformed, 
                            constants[0], constants[1], 
                            constants[2], constants[3], 
                            constants[4], constants[5], 
                            constants[6], constants[7][0], show_fig = False, verbose = False)
  scores.append(div)
  if div < best_score_tsne:
    best_score_tsne = div
    best_emb_tsne = embed_tsne
_, div = LAPS_and_plot(best_emb_tsne, X_transformed,
                constants[0], constants[1], 
                constants[2], constants[3], 
                constants[4], constants[5], 
                constants[6], constants[7][0], show_fig = True, verbose = False)
print(f'Between {all_runs} t-SNE\s best run got a score of {div}')
print(f'All the scores were {scores}')

Between 5 t-SNE\s best run got a score of 0.7765216620125726
All the scores were [0.8247961100135555, 0.7765216620125726, 0.7947571027405078, 0.8461668780210518, 0.8268521760501941]


#### fSNE (python)

In [ ]:
SNE = fSNE(X_transformed, max_iter = iterations, POI = POI, lambdas = 2)
#start = time.time()
#embed_sne = SNE.fit_transform()
best_score_fsne = 1
best_emb_fsne = 0
scores = []
for i in range(1):
  embed_fsne = SNE.fit_transform()
  _, div = LAPS_and_plot(embed_fsne, X_transformed, 
                            constants[0], constants[1], 
                            constants[2], constants[3], 
                            constants[4], constants[5], 
                            constants[6], constants[7][0], show_fig = False, verbose = False)
  scores.append(div)
  if div < best_score_fsne:
    best_score_fsne = div
    best_emb_fsne = embed_fsne
_, div = LAPS_and_plot(best_emb_fsne, X_transformed,
                constants[0], constants[1], 
                constants[2], constants[3], 
                constants[4], constants[5], 
                constants[6], constants[7][0], show_fig = True, verbose = False)
print(f'Between {all_runs} fSNE\s best run got a score of {div}')
print(f'All the scores were {scores}')

Calculating P-Matrix
Iterations 0 ...
----Cost: 298.4789036634303
Iterations 1 ...
----Cost: 296.24322774483915
Iterations 2 ...
----Cost: 294.90806333963553
Iterations 3 ...
----Cost: 293.83072383742126
Iterations 4 ...
----Cost: 292.6235279123245
Iterations 5 ...
----Cost: 290.89029192639305
Iterations 6 ...
----Cost: 288.2031399460167
Iterations 7 ...
----Cost: 284.1025328049105
Iterations 8 ...
----Cost: 278.10266017662764
Iterations 9 ...
----Cost: 269.74239005687014
Iterations 10 ...
----Cost: 258.69796939719015
Iterations 11 ...
----Cost: 244.92658473029653
Iterations 12 ...
----Cost: 228.8268437995801
Iterations 13 ...
----Cost: 211.3226843867049
Iterations 14 ...
----Cost: 193.6651195250646
Iterations 15 ...
----Cost: 177.07389691492256
Iterations 16 ...
----Cost: 162.43529254632864
Iterations 17 ...
----Cost: 150.0763396098166
Iterations 18 ...
----Cost: 139.86074219959715
Iterations 19 ...
----Cost: 131.46673281200785
Iterations 20 ...
----Cost: 124.51833616876922
Iterations

Between 5 fSNE\s best run got a score of 0.7820485479132636
All the scores were [0.7820485479132636]


In [ ]:
import multiprocessing
print()
print(multiprocessing.cpu_count())


12


In [ ]:
from multiprocessing import Pool
def test(x):
  #print(POI)
  return x**2
with Pool(5) as p:
  print(p.map(test, POI))

In [ ]:
SNE = fSNE(X_transformed, max_iter = iterations, POI = POI, lambdas = 1.25)
#start = time.time()
#embed_sne = SNE.fit_transform()
best_score_fsne = 1
best_emb_fsne = 0
scores = []
for i in range(1):
  embed_fsne = SNE.fit_transform()
  _, div = LAPS_and_plot(embed_fsne, X_transformed, 
                            constants[0], constants[1], 
                            constants[2], constants[3], 
                            constants[4], constants[5], 
                            constants[6], constants[7][0], show_fig = False, verbose = False)
  scores.append(div)
  if div < best_score_fsne:
    best_score_fsne = div
    best_emb_fsne = embed_fsne
_, div = LAPS_and_plot(best_emb_fsne, X_transformed,
                constants[0], constants[1], 
                constants[2], constants[3], 
                constants[4], constants[5], 
                constants[6], constants[7][0], show_fig = True, verbose = False)
print(f'Between {all_runs} fSNE\s best run got a score of {div}')
print(f'All the scores were {scores}')

#### fSNE (torch) WIP

In [ ]:
POI

array([277, 258, 398, 479, 187, 559, 475,   0, 183, 278, 560, 521, 149,
       201, 256, 113, 531, 334, 126, 518, 538,  70, 511,  95, 375, 325,
        71, 520, 423, 300, 100, 402,  77, 267, 141, 395, 125, 150,   0,
       176,  63, 471, 420,  46, 465, 235, 561, 348, 358, 177, 328, 146,
        67, 500, 459,  52])

In [ ]:
from focusedSNE_torch import TorchfSNE
weights = [0.5,0.5]
best_emb = 0
best_score = 1
for i in range(1):
  embed_fsne = TorchfSNE(n_iter = iterations, POI = POI, lambdas = weights).fit_transform(X_transformed)
  _, div = LAPS_and_plot(embed_fsne, X_transformed,
                constants[0], constants[1], 
                constants[2], constants[3], 
                constants[4], constants[5], 
                constants[6], constants[7][0], show_fig = False, verbose = False)
  if div < best_score:
    best_score = div
    best_emb = embed_fsne
comp, div = LAPS_and_plot(embed_fsne, X_transformed,
              constants[0], constants[1], 
              constants[2], constants[3], 
              constants[4], constants[5], 
              constants[6], constants[7][0], show_fig = True, verbose = False)
# global_comp, global_div, _ = GAPS_score(embed_fsne, constants[0], X_transformed, 
#                                         constants[2], constants[3], 
#                                         constants[4], constants[5], 
#                                         constants[6], constants[7], show_fig = False)
print(div)
# print(global_div)

0.8146845237728411


In [ ]:
from focusedSNE_torch import TorchfSNE
weights = [2,1]
best_emb = 0
best_score = 1
for i in range(1):
  embed_fsne = TorchfSNE(n_iter = iterations, POI = POI, lambdas = weights).fit_transform(X_transformed)
  _, div = LAPS_and_plot(embed_fsne, X_transformed,
                constants[0], constants[1], 
                constants[2], constants[3], 
                constants[4], constants[5], 
                constants[6], constants[7][0], show_fig = False, verbose = False)
  if div < best_score:
    best_score = div
    best_emb = embed_fsne
comp, div = LAPS_and_plot(embed_fsne, X_transformed,
              constants[0], constants[1], 
              constants[2], constants[3], 
              constants[4], constants[5], 
              constants[6], constants[7][0], show_fig = True, verbose = False)
# global_comp, global_div, _ = GAPS_score(embed_fsne, constants[0], X_transformed, 
#                                         constants[2], constants[3], 
#                                         constants[4], constants[5], 
#                                         constants[6], constants[7], show_fig = False)
print(div)
# print(global_div)

0.8305403464105499
